# 🥋 Lekcja 20: WebDataset (Format TAR dla Big Data)

Kiedy masz miliard plików, system plików staje się wąskim gardłem.
Otwarcie pliku (`open()`) trwa. Otwarcie miliarda plików trwa miliard razy dłużej.

**WebDataset (WDS)** to biblioteka i format oparty na standardowych archiwach **TAR**.
*   Zamiast: folder z 1 000 000 plików `.jpg` i `.json`.
*   Mamy: 100 plików `.tar`, a w każdym po 10 000 par (obrazek + opis).

**Zalety:**
1.  **Sekwencyjny odczyt:** Dysk czyta jeden duży ciąg bajtów (maksymalna przepustowość).
2.  **Streaming:** Możesz trenować model na danych, które leżą na S3, nie pobierając ich na dysk! (Pipe mode).
3.  **Shuffle:** Tasujemy w buforze RAM, a nie na dysku.

Zrobimy symulację: Stworzymy dataset w formacie TAR i odczytamy go strumieniowo.

In [1]:
# Instalacja WebDataset
!uv pip install webdataset

import webdataset as wds
import torch
from torch.utils.data import DataLoader
import numpy as np
import os
import shutil

# Katalog na nasze dane
DATA_DIR = "data_wds"
if os.path.exists(DATA_DIR):
    shutil.rmtree(DATA_DIR)
os.makedirs(DATA_DIR)

print(f"Katalog roboczy: {DATA_DIR}")

Resolved 4 packages in 1.19s
Prepared 2 packages in 175ms
Installed 3 packages in 55ms
 + braceexpand==0.1.7
 + pyyaml==6.0.3
 + webdataset==1.0.2


Katalog roboczy: data_wds


## Krok 1: Tworzenie Shardów (Pisanie)

Stworzymy syntetyczny dataset (obrazek + etykieta).
Zapiszemy go jako serię plików `.tar` (zwanych **Shardami**).

Użyjemy `wds.ShardWriter`.
Wzór nazwy: `dataset-%06d.tar` (dataset-000000.tar, dataset-000001.tar...).

In [2]:
# Wzorzec nazwy pliku (ograniczamy shard do 10MB lub 100 próbek)
pattern = os.path.join(DATA_DIR, "mnist-dummy-%06d.tar")

# Otwieramy pisarza
# maxcount=50: Nowy plik .tar co 50 próbek
with wds.ShardWriter(pattern, maxcount=50) as sink:
    for i in range(200): # 200 próbek total (powstaną 4 pliki tar)
        
        # Symulacja danych
        # Obrazek: Losowy tensor, zapiszemy jako bajty (np. format .pth lub surowe)
        # WDS lubi formaty standardowe (jpg, png, pyd), my użyjemy 'pth' dla tensora
        image = torch.randn(3, 32, 32)
        label = i % 10  # Klasa 0-9
        
        # Zapisujemy próbkę (Słownik)
        sample = {
            "__key__": f"sample{i:05d}",   # Unikalny klucz pliku wewnątrz tara
            "input.pth": image,            # Rozszerzenie mówi, jak to odkodować
            "label.cls": label             # .cls to format dla liczby całkowitej
        }
        
        sink.write(sample)

print("✅ Zapisano dane w formacie TAR.")
print("Lista plików:")
for f in sorted(os.listdir(DATA_DIR)):
    print(f" - {f}")

# writing data_wds\mnist-dummy-000000.tar 0 0.0 GB 0
# writing data_wds\mnist-dummy-000001.tar 50 0.0 GB 50
# writing data_wds\mnist-dummy-000002.tar 50 0.0 GB 100
# writing data_wds\mnist-dummy-000003.tar 50 0.0 GB 150
✅ Zapisano dane w formacie TAR.
Lista plików:
 - mnist-dummy-000000.tar
 - mnist-dummy-000001.tar
 - mnist-dummy-000002.tar
 - mnist-dummy-000003.tar


## Krok 2: Czytanie Strumieniowe (Pipeline)

Teraz najważniejsze. Jak to odczytać?
`wds.WebDataset` działa jak rurociąg (Pipeline) w systemie Linux.

1.  Wczytaj bajty z TAR-a.
2.  Zdekoduj (np. zamień bajty `.pth` z powrotem na Tensor).
3.  Zmień na krotkę `(input, label)`.

To wszystko dzieje się **w locie (on-the-fly)**.

In [8]:
# Generujemy listę plików ręcznie (bezpieczne na Windows/Linux)
# Zamiast wzorca "{..}", tworzymy listę konkretnych ścieżek
urls = [os.path.join(DATA_DIR, f"mnist-dummy-{i:06d}.tar") for i in range(4)]

print("Lista plików do wczytania:")
print(urls)

# Definicja Pipeline'u
# WebDataset przyjmuje listę plików równie chętnie co wzorzec
dataset = (
    wds.WebDataset(urls)      # 1. Otwórz strumień z listy
    .shuffle(100)             # 2. Tasuj w buforze (100 elementów w RAM)
    .decode()                 # 3. Automatycznie dekoduj (.pth -> Tensor, .cls -> Int)
    .to_tuple("input.pth", "label.cls") # 4. Wybierz co chcesz zwrócić
)

print("Dataset zdefiniowany (Leniwy - nic jeszcze nie wczytał).")

Lista plików do wczytania:
['data_wds\\mnist-dummy-000000.tar', 'data_wds\\mnist-dummy-000001.tar', 'data_wds\\mnist-dummy-000002.tar', 'data_wds\\mnist-dummy-000003.tar']
Dataset zdefiniowany (Leniwy - nic jeszcze nie wczytał).


## Integracja z DataLoaderem

WebDataset jest typu **IterableDataset** (pamiętasz Lekcję 15?).
Działa świetnie z `DataLoader`, ale trzeba pamiętać o batchowaniu.

WebDataset ma własną metodę `.batched(batch_size)`, która jest szybsza niż ta w DataLoaderze, bo skleja listy wewnątrz C++.

In [9]:
# Dodajemy batchowanie do pipeline'u WDS
batched_dataset = dataset.batched(16)

# DataLoader służy tu tylko do obsługi workerów i prefetchingu
# batch_size=None, bo batchowanie zrobiliśmy już wyżej w WDS!
loader = DataLoader(batched_dataset, batch_size=None, num_workers=0)

print("--- ODCZYT DANYCH ---")
for i, (imgs, labels) in enumerate(loader):
    if i == 0:
        print(f"Batch shape: {imgs.shape}")
        print(f"Labels: {labels}")
    
    # Symulacja treningu...
    pass

print(f"Przetworzono {i+1} batchy.")

--- ODCZYT DANYCH ---
Batch shape: torch.Size([16, 3, 32, 32])
Labels: tensor([7, 3, 7, 5, 3, 3, 6, 0, 1, 9, 4, 4, 0, 5, 6, 4])
Przetworzono 13 batchy.


## 🥋 Black Belt Summary

To kończy **Moduł 3: Inżynieria Danych**.

1.  **Dlaczego TAR?** System plików (OS) nie radzi sobie z milionami plików. TAR skleja je w duże bloki, co pozwala na sekwencyjny odczyt z maksymalną prędkością dysku.
2.  **WebDataset:** To standard w trenowaniu na klastrach (HPC). Pozwala na "nieskończone" zbiory danych, które nie mieszczą się na dysku lokalnym (streaming).
3.  **Struktura:** `Url -> Shuffle -> Decode -> Tuple -> Batch`.

W następnym module (**Moduł 4: Zaawansowana Architektura**) wejdziemy do środka `nn.Module`. Zrozumiemy cykl życia modelu, bufory i hooki.